# Solid ice discharge in southeast Greenland fjords

## Purpose
This notebook generates figures and spreadsheets of solid ice discharge for each glacier and fjord in the study, based on direct observations, and interpolations of those observations. The plots show the discharge time series as well as the mean annual, mean hydrological year, and mean seasonal discharge for each fjord. The spreadsheets contain cumulative and per-fjord statistics about solid ice discharge.

## Requirements
This notebook analyzes data from Mankoff _et al._ (2020) which can be downloaded here: https://doi.org/10.22008/promice/data/ice_discharge/d/v02. Wherever you locate those files, indicate the path in the user parameters below.

## Output
- Figures of observed discharge rate time series (including mean annual, _etc._) for individual glaciers: \
`../figures/glacier_icedischarge_observed/g###_obs_icedischarge_fjord##.png`
- Figures of interpolated discharge rate time series (including mean annual, _etc._) for individual glaciers: \
`../figures/glacier_icedischarge_interpolated/g###_interp_icedischarge_fjord##.png`
- Spreadsheet of cumulative discharge and mean discharge rates (based on interpolations) for individual glaciers: \
`../databases/glacier_icedischarge_interp.csv`
- Spreadsheets of monthly time series of fjord-level discharge rates (from glacier interpolations): \
`../databases/fjord_icedischarge_monthly/fjord##_icedischarge_monthly.csv`
- Figures of interpolated glacier discharge curves and total fjord discharge for each fjord: \
`../figures/fjord_icedischarge_components/fjord##_icedischarge_components.png`
- Spreadsheet of cumulative discharge and mean discharge rates for each fjord: \
`../databases/fjord_icedischarge_interp.csv`

## Code

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pathlib

In [ ]:
# SET USER PARAMETERS
startdate = '2015-01-01'
enddate = '2019-12-31'
threshold = 0.5 # coverage threshold for solid ice discharge, used by Mankoff
solid_ice_dir = '../IceDischarge_Mankoff/' # local path to files downloaded from https://doi.org/10.22008/promice/data/ice_discharge/d/v02

In [ ]:
# READ FILES

# Read in file that relates Moon pointID, Joughin glacierID, and Mankoff gate ID
id_map = pd.read_csv('./relate_pointID_glacierID.txt', delimiter=' ')
id_map.set_index('pointID', inplace=True)

# Read in file that relates glacier ID to fjord number
glacier_fjord_db = pd.read_csv('./glaciers_fjords.txt', delimiter=',')

# Read in solid ice discharge time series at each gate
gate_discharge = pd.read_csv(f'{solid_ice_dir}/gate_D.csv')
gate_discharge.set_index('Date', inplace=True)

# Read in error in solid ice discharge time series at each gate
gate_error = pd.read_csv(f'{solid_ice_dir}/gate_err.csv')
gate_error.set_index('Date', inplace=True)

# Read in coverage in solid ice discharge time series at each gate
gate_coverage = pd.read_csv(f'{solid_ice_dir}/gate_coverage.csv')
gate_coverage.set_index('Date', inplace=True)

In [ ]:
# DEFINE FUNCTIONS

def getMankoffID(pointID, id_map):
    """Find Mankoff gate ID associated with Moon glacier point ID"""
    mankoffID = id_map.loc[pointID].mankoffID
    if mankoffID is not np.nan:
        mankoffID = mankoffID.split(',')
    return mankoffID

def timeSeries(mankoffID, gate_discharge, gate_error, gate_coverage):
    """Create single dataframe with time series of discharge, error, and coverage at a Mankoff gate"""
    discharge = gate_discharge[mankoffID]
    error = gate_error[mankoffID]
    coverage = gate_coverage[mankoffID]
    data = pd.DataFrame(index=pd.to_datetime(discharge.index), data={'discharge': discharge.values.flatten(), 'error': error.values.flatten(), 'coverage': coverage.values.flatten()})
    return data

def filterDates(data, startdate, enddate):
    """Filter data to samples between start and end dates (inclusive)"""
    data = data[data.index >= startdate]
    data = data[data.index <= enddate]
    return data

def filterCoverage(data, threshold):
    """Filter data to points at or above a given threshold coverage"""
    data = data[data.coverage >= threshold]
    return data

def rmse(error_data):
    """Calculate root mean squared error (RMSE) when calculating mean of data"""
    if len(error_data.dropna()) == 0:
        rmse = np.nan
    else: 
        error_data = error_data.dropna()
        rmse = sum([error**2 for error in error_data])**0.5 / len(error_data)**0.5
    return rmse

def interpolation_error(data):
    """Calculate interpolation error associated with interpolating between discharge values in a dataset. 
    Dataset must be dataframe containing 'discharge' and 'error' columns."""
    # get dates of time series
    data_index = data.index
    # create a copy of data so as not to overwrite original data variable
    data_filled = data.copy()
    # loop from first valid (notnull) date to last valid date - cannot interpolate outside this range
    for x in data_index[(data_index >= data.first_valid_index()) & (data_index <= data.last_valid_index())]:
        if data.loc[x].isna().error: # if error is null (value is interpolated)
            # prior valid (notnull) data
            x1 = data[data_index < x].last_valid_index()
            y1 = data.loc[x1].discharge
            u1 = data.loc[x1].error
            # next valid (notnull) data
            x2 = data[data_index > x].first_valid_index()
            y2 = data.loc[x2].discharge
            u2 = data.loc[x2].error
            # calculate interpolation error (from DOI:10.1007/s10765-016-2174-6, Eqn 15)
            u = np.sqrt((((x-x2)/(x1-x2))**2)*(u1**2) + (((x-x1)/(x2-x1))**2)*(u2**2))
            data_filled.loc[x].error = u
    return data_filled

def interpolate_discharge(data, date_index):
    """Interpolate across discharge observations and calculate interpolation error."""
    if data is np.nan: # data is empty (no discharge for this glacier)
        data_filled = data # just return same nan
    elif isinstance(data, pd.DataFrame): # data represents one glacier
            # resample sparse discharge data to a daily grid
        data_daily = data.reindex(date_index)
        # calculate interpolation error between sparse discharge data points
        data_interr = interpolation_error(data_daily)
        # linearly interpolate to fill gaps (missing days), and apply a 120-day rolling mean
        data_filled = data_interr.interpolate(method='linear', limit_area='inside').rolling(window='120D', center=True).agg({'discharge': 'mean', 'error': rmse})
    else: # data represents more than one glacier
        # resample sparse discharge data to a daily grid
        data_daily = [x.reindex(date_index) for x in data]
        # calculate interpolation error between sparse discharge data points
        data_interr = [interpolation_error(x) for x in data_daily]
        # linearly interpolate to fill gaps (missing days), and apply a 120-day rolling mean
        data_filled = [x.interpolate(method='linear', limit_area='inside').rolling(window='120D', center=True).agg({'discharge': 'mean', 'error': rmse}) for x in 
        data_interr]
    return data_filled

def season_aggregate(dates):
    """Output seasons from list of dates. December is rolled over to following year."""
    def get_season(date):
        if date.month in [1, 2]:
            season = '{}-01'.format(date.year)
        elif date.month in [3, 4, 5]:
            season = '{}-04'.format(date.year)
        elif date.month in [6, 7, 8]:
            season = '{}-07'.format(date.year)
        elif date.month in [9, 10, 11]:
            season = '{}-10'.format(date.year)
        elif date.month in [12]:
            season = '{}-01'.format(date.year+1)
        return season
    seasons = pd.to_datetime([get_season(date) for date in dates], format='%Y-%m')
    return seasons

def hydroyear_aggregate(dates):
    """Output hydrological year (September start) from list of dates"""
    def get_hydroyear(date):
        if date.month >= 9:
            hydroyear = '{}-03'.format(date.year+1)
        elif date.month < 9:
            hydroyear = '{}-03'.format(date.year)
        return hydroyear
    hydroyears = pd.to_datetime([get_hydroyear(date) for date in dates], format='%Y-%m')
    return hydroyears

def getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold):
    """Get gate discharge, error, and coverage for a point ID and filter by dates and coverage. Returns pandas dataframe or nan."""
    mankoff = getMankoffID(pointID, id_map)
    if mankoff is not np.nan:
        if len(mankoff) == 1:
            data = timeSeries(mankoff, gate_discharge, gate_error, gate_coverage)
            data = filterDates(data, startdate, enddate)
            data = filterCoverage(data, threshold)
        else:
            gates_data = [timeSeries(id, gate_discharge, gate_error, gate_coverage) for id in mankoff]
            gates_data = [filterDates(df, startdate, enddate) for df in gates_data]
            gates_data = [filterCoverage(df, threshold) for df in gates_data]
            combined_discharge = sum([df.discharge for df in gates_data])
            combined_error = sum([df.error**2 for df in gates_data])**0.5
            combined_coverage = sum([df.coverage for df in gates_data])/len(gates_data)
            data = pd.DataFrame(index=combined_discharge.index, data={'discharge': combined_discharge, 'error': combined_error, 'coverage': combined_coverage})
            data = data.dropna()
    else:
        print('No discharge data associated with glacier point {}'.format(pointID))
        data = np.nan
    return data

def meanAnnualDischarge(data):
    """Calculate mean annual discharge from full time series of data (from getDischargeData)"""
    if data is not np.nan:
        mean_annual_data = data.groupby(data.index.year).agg({'discharge': 'mean', 'error': rmse})
        mean_annual_data.index = pd.to_datetime(['{}-07'.format(y) for y in mean_annual_data.index], format='%Y-%m')
    else: mean_annual_data = np.nan
    return mean_annual_data

def meanHydroAnnualDischarge(data):
    """Calculate mean hydrological year discharge from full time series of data (from getDischargeData) (hydroyear starts September)"""
    if data is not np.nan:
        hydroyear_data = data.copy()
        hydroyear_data['hydroyear'] = hydroyear_aggregate(hydroyear_data.index)
        mean_hydroyear_data = hydroyear_data.groupby('hydroyear').agg({'discharge': 'mean', 'error': rmse})
    else: mean_hydroyear_data = np.nan
    return mean_hydroyear_data

def meanSeasonalDischarge(data):
    """Calculate mean seasonal discharge from full time series of data (from getDischargeData)"""
    if data is not np.nan:
        seasonal_data = data.copy()
        seasonal_data['season'] = season_aggregate(seasonal_data.index)
        mean_seasonal_data = seasonal_data.groupby('season').agg({'discharge': 'mean', 'error': rmse})
    else: mean_seasonal_data = np.nan
    return mean_seasonal_data

def getFjordNumber(pointID):
    """Get fjord number associated with a given glacier point ID"""
    fjord_number = int(glacier_fjord_db.where(glacier_fjord_db['GlacierID'] == pointID)['FjordID'].dropna().values)
    return fjord_number

def getFjordGlaciers(fjordID):
    """Get glacier point IDs associated with a given fjord number"""
    glacierIDs = [int(x) for x in glacier_fjord_db.where(glacier_fjord_db['FjordID'] == fjordID)['GlacierID'].dropna().values]
    return glacierIDs

def plotGlacierDischarge(ax, data, pointID):
    """Plot solid ice discharge time series for a glacier (given by point ID)"""
    if data is not np.nan:
        ax.plot(data.index, data.discharge, '.-')
        ax.fill_between(data.index, data.discharge+data.error, data.discharge-data.error, alpha=0.2, label='error')
        ax.set_xlabel('Time')
        ax.set_ylabel('Discharge [Gt yr$^{-1}$]')
        ax.set_title(f'Glacier {pointID} discharge\n(Fjord {getFjordNumber(pointID)})')
        ax.grid('on')

def plotFjordDischarge(ax, fjord_data, fjordID):
    """Plot solid ice discharge time series for a fjord (given by fjord ID)"""
    if fjord_data is not np.nan:
        ax.plot(fjord_data.index, fjord_data.discharge, '.-')
        ax.fill_between(fjord_data.index, fjord_data.discharge + fjord_data.error, fjord_data.discharge - fjord_data.error, alpha=0.2, label='error')
        ax.set_xlabel('Time')
        ax.set_ylabel('Discharge [Gt yr$^{-1}$]')
        ax.set_title(f'Fjord {fjordID} discharge')
        ax.grid('on')


### Discharge time series for individual glaciers
Plot time series including all observations, mean annual, mean hydrological year, and mean seasonal discharge rates.

In [ ]:
# PLOT INDIVIDUAL GLACIER DISCHARGE RATE OBSERVATIONS WITH ERROR
pointID = 147
data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate=startdate, enddate=enddate, threshold=threshold)
if data is not np.nan:
    fig, ax = plt.subplots()
    plotGlacierDischarge(ax, data, pointID)
    ax.set_xticks(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01'])
    ax.set_xticklabels(['2015', '2016', '2017', '2018', '2019', '2020'])
    plt.show()

In [ ]:
# PLOT MEAN ANNUAL DISCHARGE RATE
pointID = 147
data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)
mean_annual_data = meanAnnualDischarge(data)

if mean_annual_data is not np.nan:
    fig, ax = plt.subplots()
    plotGlacierDischarge(ax, mean_annual_data, pointID)
    ax.set_xticks(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01'])
    ax.set_xticklabels(['2015', '2016', '2017', '2018', '2019'])
    ax.set_title('Glacier #{} mean annual discharge'.format(pointID))
    plt.show()

In [ ]:
# PLOT MEAN HYDROLOGICAL YEAR DISCHARGE RATE
pointID = 147
data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)
mean_hydroyear_data = meanHydroAnnualDischarge(data)
if mean_hydroyear_data is not np.nan:
    fig, ax = plt.subplots()
    plotGlacierDischarge(ax, mean_hydroyear_data, pointID)
    ax.set_xticks(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01'])
    ax.set_xticklabels(['2015', '2016', '2017', '2018', '2019', '2020'])
    ax.set_title('Glacier #{} mean hydrological year discharge'.format(pointID))
    plt.show()

In [ ]:
# PLOT MEAN SEASONAL DISCHARGE RATE
pointID = 147
data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)
mean_seasonal_data = meanSeasonalDischarge(data)
if mean_seasonal_data is not np.nan:
    fig, ax = plt.subplots()
    plotGlacierDischarge(ax, mean_seasonal_data, pointID)
    ax.set_xticks(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01'])
    ax.set_xticklabels(['2015', '2016', '2017', '2018', '2019', '2020'])
    ax.set_title('Glacier #{} mean seasonal discharge'.format(pointID))
    plt.show()

In [ ]:
# COMBINE ABOVE PLOTS INTO ONE
pointID = 175
data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)
mean_annual_data = meanAnnualDischarge(data)
mean_hydroyear_data = meanHydroAnnualDischarge(data)
mean_seasonal_data = meanSeasonalDischarge(data)

if data is not np.nan:
    fig, ax = plt.subplots()
    plotGlacierDischarge(ax, data, pointID)
    ax.set_title(f'Observed discharge\nGlacier {pointID}, Fjord {getFjordNumber(pointID)}')
    mean_seasonal_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
    mean_annual_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
    mean_hydroyear_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
    ax.grid('on')
    ax.legend(handles=ax.get_lines(), labels=['discharge', 'seasonal mean', 'annual mean', 'hydrological year mean'], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
# PLOT DISCHARGE RATE OBSERVATIONS/MEANS TIME SERIES FOR ALL GLACIERS AND SAVE
pathlib.Path('../figures/glacier_icedischarge_observed').mkdir(parents=True, exist_ok=True)
for pointID in id_map.index:
    data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)

    if data is not np.nan:
        mean_annual_data = meanAnnualDischarge(data)
        mean_hydroyear_data = meanHydroAnnualDischarge(data)
        mean_seasonal_data = meanSeasonalDischarge(data)

        fig, ax = plt.subplots()
        plotGlacierDischarge(ax, data, pointID)
        ax.set_title(f'Observed discharge\nGlacier {pointID}, Fjord {getFjordNumber(pointID)}')
        mean_seasonal_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
        mean_annual_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
        mean_hydroyear_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
        ax.grid('on')
        ax.legend(handles=ax.get_lines(), labels=['discharge', 'seasonal mean', 'annual mean', 'hydrological year mean'], loc='center left', bbox_to_anchor=(1, 0.5))

        plt.savefig('../figures/glacier_icedischarge_observed/g{:03}_obs_icedischarge_fjord{:02}.png'.format(pointID, getFjordNumber(pointID)), bbox_inches='tight', dpi=300)
        plt.close()

### Interpolated discharge time series for individual glaciers
Interpolate glacier observations and plot time series including interpolated discharge, mean annual, mean hydrological year, and mean seasonal discharge rates.

In [ ]:
# GLACIER DISCHARGE INTERPOLATION SCHEME
# We resample the sparse discharge rate data to a daily grid, linearly interpolate to fill gaps (missing days), and apply a 120-day rolling mean
gid = 175
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))
data = getDischargeData(gid, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)

if data is not np.nan:
    interpolated_data = interpolate_discharge(data, daily_index)
    # plot discharge observations and daily interpolated curve
    fig, ax = plt.subplots()
    ax.errorbar(x=data.index, y=data.discharge, yerr=data.error, color='darkgray', fmt='.', label='observations')
    ax.errorbar(interpolated_data.index, interpolated_data.discharge, color='dodgerblue', linewidth=3, label='daily interp')
    ax.fill_between(x=interpolated_data.index, y1=interpolated_data.discharge-interpolated_data.error, y2=interpolated_data.discharge+interpolated_data.error, color='dodgerblue', alpha=0.5)
    ax.set_title(f'Glacier {gid}')
    ax.set_ylabel('Solid ice discharge (Gt/yr)')
    ax.legend()
    plt.show()

In [ ]:
# REPEAT PREVIOUS INDIVIDUAL OBSERVATIONS AND MEANS COMBINED PLOT, WITH INTERPOLATION
pointID = 175

# get observations and interpolate
data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))

if data is not np.nan:
    interpolated_data = interpolate_discharge(data, daily_index)
    # calculate means over interpolated data
    mean_annual_data = meanAnnualDischarge(interpolated_data)
    mean_hydroyear_data = meanHydroAnnualDischarge(interpolated_data)
    mean_seasonal_data = meanSeasonalDischarge(interpolated_data)

    # plot interpolations
    fig, ax = plt.subplots()
    plotGlacierDischarge(ax, interpolated_data, pointID)
    ax.set_title(f'Interpolated discharge\nGlacier {pointID}, Fjord {getFjordNumber(pointID)}')
    mean_seasonal_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
    mean_annual_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
    mean_hydroyear_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
    ax.grid('on')
    ax.legend(handles=ax.get_lines(), labels=['discharge', 'seasonal mean', 'annual mean', 'hydrological year mean'], loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
# PLOT DISCHARGE RATE INTERPOLATIONS/MEANS TIME SERIES FOR ALL GLACIERS AND SAVE
pathlib.Path('../figures/glacier_icedischarge_interpolated').mkdir(parents=True, exist_ok=True)
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))
for pointID in id_map.index:
    data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold)

    if data is not np.nan:
        # interpolate the observed data
        interpolated_data = interpolate_discharge(data, daily_index)
        # calculate means
        mean_annual_data = meanAnnualDischarge(interpolated_data)
        mean_hydroyear_data = meanHydroAnnualDischarge(interpolated_data)
        mean_seasonal_data = meanSeasonalDischarge(interpolated_data)

        fig, ax = plt.subplots()
        plotGlacierDischarge(ax, interpolated_data, pointID)
        ax.set_title(f'Interpolated discharge\nGlacier {pointID}, Fjord {getFjordNumber(pointID)}')
        mean_seasonal_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
        mean_annual_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
        mean_hydroyear_data.discharge.plot(ax=ax, style='.-', alpha=0.8)
        ax.grid('on')
        ax.legend(handles=ax.get_lines(), labels=['discharge', 'seasonal mean', 'annual mean', 'hydrological year mean'], loc='center left', bbox_to_anchor=(1, 0.5))

        plt.savefig('../figures/glacier_icedischarge_interpolated/g{:03}_interp_icedischarge_fjord{:02}.png'.format(pointID, getFjordNumber(pointID)), bbox_inches='tight', dpi=300)
        plt.close()

### Cumulative and mean discharge for each glacier
Calculate cumulative discharge over the time series, and mean annual/hydrological year/seasonal discharge rates. Output to spreadsheet.

In [ ]:
discharge_data = pd.DataFrame(index=id_map.index, columns=[
    'Fjord number', 
    'Cumulative discharge (Gt)', 'Cumulative error (Gt)', 
    'Mean annual discharge rate (Gt/yr)', 'Mean annual discharge rate error (Gt/yr)', 
    'Mean hydroyear discharge rate (Gt/yr)', 'Mean hydroyear discharge rate error (Gt/yr)', 
    'Mean winter discharge rate (Gt/yr)', 'Mean winter discharge rate error (Gt/yr)', 
    'Mean spring discharge rate (Gt/yr)', 'Mean spring discharge rate error (Gt/yr)', 
    'Mean summer discharge rate (Gt/yr)', 'Mean summer discharge rate error (Gt/yr)', 
    'Mean autumn discharge rate (Gt/yr)', 'Mean autumn discharge rate error (Gt/yr)'])
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))

for pointID in id_map.index:

    # Fjord number
    fjord_number = getFjordNumber(pointID)
    fjord_number = int(fjord_number)

    # Get glacier discharge data
    data = getDischargeData(pointID, id_map, gate_discharge, gate_error, gate_coverage, startdate=startdate, enddate=enddate, threshold=threshold)
    if data is not np.nan: # if glacier has discharge, interpolate and calculate cumulative/means
        interpolated_data = interpolate_discharge(data, daily_index)
        cumulative_discharge = interpolated_data.agg({'discharge': 'sum', 'error': 'sum'}) / 365 # discharge data is rate on a day. [Gt/yr]*[1 yr/365 d]*[1 d] = Gt
        # mean annual
        mean_annual_data = meanAnnualDischarge(interpolated_data)
        mean_annual_mean = mean_annual_data.agg({'discharge': 'mean', 'error': rmse})
        # mean hydrological year
        mean_hydroyear_data = meanHydroAnnualDischarge(interpolated_data)
        mean_hydroyear_mean = mean_hydroyear_data.agg({'discharge': 'mean', 'error': rmse})
        # mean seasonal
        mean_seasonal_data = meanSeasonalDischarge(interpolated_data)
        mean_seasonal_mean = mean_seasonal_data.groupby(mean_seasonal_data.index.month).agg({'discharge': 'mean', 'error': rmse})
    else: # otherwise, make all attributes nan for this glacier
        cumulative_discharge = pd.DataFrame(index=[pointID], data={'discharge': np.nan, 'error': np.nan})
        mean_annual_mean = pd.DataFrame(index=[pointID], data={'discharge': np.nan, 'error': np.nan})
        mean_hydroyear_mean = pd.DataFrame(index=[pointID], data={'discharge': np.nan, 'error': np.nan})
        mean_seasonal_mean = pd.DataFrame(index=[1, 4, 7, 10], columns=['discharge', 'error'])

    # Compile dataframe of all means and errors
    mean_data = pd.DataFrame(index=[pointID], data={
        'Fjord number': fjord_number,
        'Cumulative discharge (Gt)': cumulative_discharge.discharge,
        'Cumulative error (Gt)': cumulative_discharge.error,
        'Mean annual discharge rate (Gt/yr)': mean_annual_mean.discharge,
        'Mean annual discharge rate error (Gt/yr)': mean_annual_mean.error,
        'Mean hydroyear discharge rate (Gt/yr)': mean_hydroyear_mean.discharge,
        'Mean hydroyear discharge rate error (Gt/yr)': mean_hydroyear_mean.error,
        'Mean winter discharge rate (Gt/yr)': mean_seasonal_mean.loc[1].discharge,
        'Mean winter discharge rate error (Gt/yr)': mean_seasonal_mean.loc[1].error,
        'Mean spring discharge rate (Gt/yr)': mean_seasonal_mean.loc[4].discharge,
        'Mean spring discharge rate error (Gt/yr)': mean_seasonal_mean.loc[4].error,
        'Mean summer discharge rate (Gt/yr)': mean_seasonal_mean.loc[7].discharge,
        'Mean summer discharge rate error (Gt/yr)': mean_seasonal_mean.loc[7].error,
        'Mean autumn discharge rate (Gt/yr)': mean_seasonal_mean.loc[10].discharge,
        'Mean autumn discharge rate error (Gt/yr)': mean_seasonal_mean.loc[10].error
    })

    # Input data into main dataframe
    discharge_data.loc[pointID] = mean_data.values

discharge_data['Fjord number'] = discharge_data['Fjord number'].astype('int')

pathlib.Path('../databases').mkdir(parents=True, exist_ok=True)
discharge_data.to_csv('../databases/glacier_icedischarge_interp.csv', na_rep='NaN')

### Calculate fjord-level discharge by interpolating and combining glacier discharges

In [ ]:
# ADD INTERPOLATION CURVES TOGETHER TO GET TOTAL SOLID ICE DISCHARGE IN A FJORD
f = 13
glacier_id_list = getFjordGlaciers(f)
print(f'Glaciers in fjord: {glacier_id_list}')
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))
if not np.isnan(glacier_id_list).all(): # if fjord contains glaciers
    # get discharge data for each glacier
    data = [getDischargeData(id, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold) for id in glacier_id_list]
    # get rid of glaciers with no discharge data
    data = [x for x in data if isinstance(x, pd.DataFrame)]
    if data: # if data exists (e.g. if glacier discharge is not empty)
        # interpolate discharge data
        interpolated_data = interpolate_discharge(data, daily_index)
        # combine discharge and error measurements across all glaciers to get values for the fjord
        fjord_discharge = sum([df.discharge for df in interpolated_data]) # add discharges together
        fjord_error = np.sqrt(sum([df.error**2 for df in interpolated_data])) # root of sum of squares of errors
        fjord_data = pd.concat([fjord_discharge, fjord_error], axis=1)

        fig, ax = plt.subplots()
        [ax.plot(df.index, df.discharge, linestyle='--') for df in interpolated_data]
        [ax.fill_between(x=df.index, y1=df.discharge-df.error, y2=df.discharge+df.error, alpha=0.3) for df in interpolated_data]
        ax.plot(fjord_data.discharge, color='black', linewidth=3)
        ax.fill_between(x=fjord_data.index, y1=fjord_data.discharge-fjord_data.error, y2=fjord_data.discharge+fjord_data.error, color='gray', alpha=0.3)
        ax.set_ylabel('Solid ice discharge [Gt yr$^{-1}$]')
        ax.set_title(f'Fjord {f}')
        ax.legend(handles=ax.get_lines()[-2:], labels=['individual glaciers', 'fjord total'], loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()

#### Get combined discharge for each fjord
Calculate a discharge curve for each glacier in a fjord and add the glaciers together to get total discharge in a fjord.

Save output for individual fjords to CSV and as plots.

Calculate cumulative, mean annual, mean hydroyear, and mean seasonal discharges at the fjord level and save to CSV.

In [ ]:
# CALCULATE DISCHARGE FOR INDIVIDUAL FJORDS, SAVE TO CSV AND PLOT

# get list of fjord ID numbers
fjord_list = sorted(glacier_fjord_db['FjordID'].unique())

# ensure save-to paths exist
pathlib.Path('../databases/fjord_icedischarge_monthly').mkdir(parents=True, exist_ok=True)
pathlib.Path('../figures/fjord_icedischarge_components').mkdir(parents=True, exist_ok=True)

# generate DF index, daily for interpolation
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))

# calculate combined solid ice discharge in each fjord, save to CSV, and plot
for f in fjord_list:
    print(f'Fjord {f}:')
    # get list of glacier ids in this fjord
    glacier_id_list = getFjordGlaciers(f)
    if np.isnan(glacier_id_list).all():
        print(f'No glaciers associated with Fjord {f} in Glacier Database XLSX.')
    elif not np.isnan(glacier_id_list).all(): # if fjord contains glaciers
        print(f'Glaciers: {glacier_id_list}')
        # get discharge data for each glacier
        data = [getDischargeData(id, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold) for id in glacier_id_list]
        # get rid of glaciers with no discharge data
        data = [x for x in data if isinstance(x, pd.DataFrame)]
        if data: # if data exists (i.e. if at least one glacier has discharge data)
            # interpolate discharge data
            interpolated_data = interpolate_discharge(data, daily_index)
            # combine discharge and error measurements across all glaciers to get values for the fjord
            fjord_discharge = sum([df.discharge for df in interpolated_data]) # add discharges together
            fjord_error = np.sqrt(sum([df.error**2 for df in interpolated_data])) # root of sum of squares of errors
            fjord_data = pd.concat([fjord_discharge, fjord_error], axis=1)
            # sample fjord discharge at monthly frequency (calculate mean for month) and report index at the 15th of the month
            monthly_fjord_data = fjord_data.resample('MS').agg({'discharge': 'mean', 'error': rmse})
            monthly_fjord_data.index = monthly_fjord_data.index + pd.tseries.frequencies.to_offset('14D')
            
            # save monthly fjord discharge to CSV
            monthly_fjord_data.to_csv(
                f'../databases/fjord_icedischarge_monthly/fjord{f:02}_icedischarge_monthly.csv', 
                na_rep='NaN', header=['Discharge (Gt/yr)', 'Error (Gt/yr)'], index=True, index_label='Date')
            
            # plot fjord total and individual glacier discharge and save figure
            # -- string cleaning for title
            if len(glacier_id_list) == 1:
                glacier_str = '(glacier ' + str([int(x) for x in glacier_id_list]).strip('[]') + ')'
            elif len(glacier_id_list) > 1:
                glacier_str = '(glaciers ' + str([int(x) for x in glacier_id_list]).strip('[]') + ')'
            # -- create plot
            fig, ax = plt.subplots()
            # -- plot discharge and error for individual glaciers
            [ax.plot(df.index, df.discharge, linestyle='--') for df in interpolated_data]
            [ax.fill_between(x=df.index, y1=df.discharge-df.error, y2=df.discharge+df.error, alpha=0.3) for df in interpolated_data]
            # -- plot discharge and error for whole fjord
            ax.plot(fjord_data.discharge, color='black', linewidth=3)
            ax.fill_between(x=fjord_data.index, y1=fjord_data.discharge-fjord_data.error, y2=fjord_data.discharge+fjord_data.error, color='gray', alpha=0.3)
            # -- figure decorations
            ax.set_title(f'Fjord {f} solid ice discharge\n{glacier_str}')
            ax.set_xlabel('Year')
            ax.set_xticks(pd.to_datetime(['2015-01-01', '2016-01-01', '2017-01-01', '2018-01-01', '2019-01-01', '2020-01-01']))
            ax.set_xticklabels(['2015', '2016', '2017', '2018', '2019', '2020'], rotation=0, horizontalalignment='center')
            ax.set_ylabel('Daily solid ice discharge [Gt yr$^{-1}$]')
            ax.set_ylim(bottom=0)
            ax.grid('on')
            ax.legend(handles=ax.get_lines()[-2:], labels=['individual glaciers', 'fjord total'], loc='center left', bbox_to_anchor=(1, 0.5))
            # -- save and close
            plt.savefig(f'../figures/fjord_icedischarge_components/fjord{f:02}_icedischarge_components.png', bbox_inches='tight', dpi=300)
            plt.close()


In [ ]:
# CALCULATE CUMULATIVE, ANNUAL MEAN, HYDROYEAR MEAN, SEASONAL MEANS FOR A FJORD
f = 13
glacier_id_list = glacier_fjord_db[glacier_fjord_db['FjordID'] == f]['GlacierID'].values
print(f'Glaciers in fjord: {glacier_id_list}')
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))
if not np.isnan(glacier_id_list).all(): # if fjord contains glaciers
    # get discharge data for each glacier
    data = [getDischargeData(id, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold) for id in glacier_id_list]
    # get rid of glaciers with no discharge data
    data = [x for x in data if isinstance(x, pd.DataFrame)]
    if data: # if data exists (e.g. if glacier discharge is not empty)
        # interpolate discharge data
        interpolated_data = interpolate_discharge(data, daily_index)
        # combine discharge and error measurements across all glaciers to get values for the fjord
        fjord_discharge = sum([df.discharge for df in interpolated_data]) # add discharges together
        fjord_error = np.sqrt(sum([df.error**2 for df in interpolated_data])) # root of sum of squares of errors
        fjord_data = pd.concat([fjord_discharge, fjord_error], axis=1)

        # calculate means
        meanann = meanAnnualDischarge(fjord_data)
        meanhyd = meanHydroAnnualDischarge(fjord_data)
        meanseas = meanSeasonalDischarge(fjord_data)

        # plot mean discharges
        fig, ax = plt.subplots()
        plotFjordDischarge(ax, fjord_data, f)
        meanseas.discharge.plot(ax=ax, style='.-', alpha=0.8)
        meanann.discharge.plot(ax=ax, style='.-', alpha=0.8)
        meanhyd.discharge.plot(ax=ax, style='.-', alpha=0.8)
        ax.grid('on')
        ax.legend(handles=ax.get_lines(), labels=['discharge', 'seasonal mean', 'annual mean', 'hydrological year mean'], loc='center left', bbox_to_anchor=(1, 0.5))
        plt.show()

In [ ]:
# CALCULATE CUMULATIVE, ANNUAL MEAN, HYDROYEAR MEAN, SEASONAL MEANS FOR A FJORD AND ENTER INTO CSV

# Create dataframe structure for saving cumulative and mean data for each fjord
fjord_list = sorted(glacier_fjord_db['FjordID'].unique())
fjord_discharge_data = pd.DataFrame(index=fjord_list, columns=[
    'Glacier numbers', 
    'Cumulative discharge (Gt)', 'Cumulative error (Gt)', 
    'Mean annual discharge rate (Gt/yr)', 'Mean annual discharge rate error (Gt/yr)', 
    'Mean hydroyear discharge rate (Gt/yr)', 'Mean hydroyear discharge rate error (Gt/yr)', 
    'Mean winter discharge rate (Gt/yr)', 'Mean winter discharge rate error (Gt/yr)', 
    'Mean spring discharge rate (Gt/yr)', 'Mean spring discharge rate error (Gt/yr)', 
    'Mean summer discharge rate (Gt/yr)', 'Mean summer discharge rate error (Gt/yr)', 
    'Mean autumn discharge rate (Gt/yr)', 'Mean autumn discharge rate error (Gt/yr)'])

# Create daily index for recalculating glacier discharges
daily_index = pd.DatetimeIndex(pd.date_range(start=startdate, end=enddate, freq='D'))

for fjordID in fjord_list:
    print(f'Processing Fjord {fjordID}')

    # Glacier numbers
    glacier_id_list = getFjordGlaciers(fjordID)
    print(f'Glaciers in fjord {fjordID}: {glacier_id_list}')

    if not np.isnan(glacier_id_list).all(): # if fjord contains glaciers
        # get discharge data for each glacier
        data = [getDischargeData(id, id_map, gate_discharge, gate_error, gate_coverage, startdate, enddate, threshold) for id in glacier_id_list]
        # get rid of glaciers with no discharge data
        data = [x for x in data if isinstance(x, pd.DataFrame)]
        if data: # if data exists (e.g. if glacier discharge is not empty)
            # interpolate discharge data
            interpolated_data = interpolate_discharge(data, daily_index)
            # combine discharge and error measurements across all glaciers to get values for the fjord
            fjord_discharge = sum([df.discharge for df in interpolated_data]) # add discharges together
            fjord_error = np.sqrt(sum([df.error**2 for df in interpolated_data])) # root of sum of squares of errors
            fjord_data = pd.concat([fjord_discharge, fjord_error], axis=1)

            if fjord_data is not np.nan: # if fjord has discharge, calculate cumulative/means
                # cumulative discharge
                # -- divide by 365: original discharge values are rates. [Gt/yr]*[1 yr/365 d]*[1 d] = Gt
                fjord_cumulative_discharge = fjord_data.agg({'discharge': 'sum', 'error': 'sum'}) / 365
                # mean annual
                fjord_mean_annual = meanAnnualDischarge(fjord_data)
                fjord_mean_annual_mean = fjord_mean_annual.agg({'discharge': 'mean', 'error': rmse})
                # mean hydrological year
                fjord_mean_hydroyear = meanHydroAnnualDischarge(fjord_data)
                fjord_mean_hydroyear_mean = fjord_mean_hydroyear.agg({'discharge': 'mean', 'error': rmse})
                # mean seasonal
                fjord_mean_seasonal = meanSeasonalDischarge(fjord_data)
                fjord_mean_seasonal_mean = fjord_mean_seasonal.groupby(fjord_mean_seasonal.index.month).agg({'discharge': 'mean', 'error': rmse})
            else: # otherwise, make all attributes nan for this fjord
                fjord_cumulative_discharge = pd.DataFrame(index=[fjordID], data={'discharge': np.nan, 'error': np.nan})
                fjord_mean_annual_mean = pd.DataFrame(index=[fjordID], data={'discharge': np.nan, 'error': np.nan})
                fjord_mean_hydroyear_mean = pd.DataFrame(index=[fjordID], data={'discharge': np.nan, 'error': np.nan})
                fjord_mean_seasonal_mean = pd.DataFrame(index=[1, 4, 7, 10], columns=['discharge', 'error'])

            # Compile dataframe of all means and errors
            fjord_mean_data = pd.DataFrame(index=[fjordID], data={
                'Glacier numbers': str(glacier_id_list),
                'Cumulative discharge (Gt)': fjord_cumulative_discharge.discharge,
                'Cumulative error (Gt)': fjord_cumulative_discharge.error,
                'Mean annual discharge rate (Gt/yr)': fjord_mean_annual_mean.discharge,
                'Mean annual discharge rate error (Gt/yr)': fjord_mean_annual_mean.error,
                'Mean hydroyear discharge rate (Gt/yr)': fjord_mean_hydroyear_mean.discharge,
                'Mean hydroyear discharge rate error (Gt/yr)': fjord_mean_hydroyear_mean.error,
                'Mean winter discharge rate (Gt/yr)': fjord_mean_seasonal_mean.loc[1].discharge,
                'Mean winter discharge rate error (Gt/yr)': fjord_mean_seasonal_mean.loc[1].error,
                'Mean spring discharge rate (Gt/yr)': fjord_mean_seasonal_mean.loc[4].discharge,
                'Mean spring discharge rate error (Gt/yr)': fjord_mean_seasonal_mean.loc[4].error,
                'Mean summer discharge rate (Gt/yr)': fjord_mean_seasonal_mean.loc[7].discharge,
                'Mean summer discharge rate error (Gt/yr)': fjord_mean_seasonal_mean.loc[7].error,
                'Mean autumn discharge rate (Gt/yr)': fjord_mean_seasonal_mean.loc[10].discharge,
                'Mean autumn discharge rate error (Gt/yr)': fjord_mean_seasonal_mean.loc[10].error
            })

            # Input data into main dataframe
            fjord_discharge_data.loc[fjordID] = fjord_mean_data.values

pathlib.Path('../databases').mkdir(parents=True, exist_ok=True)
fjord_discharge_data.to_csv('../databases/fjord_icedischarge_interp.csv', na_rep='NaN', index=True, index_label='Fjord number')